# Query ESGF input4MIPs index and extract source_id entries

## imports

In [39]:
import json
import numpy as np
import requests

## example source_id queries

## get source_id entries

In [49]:
#input4MIPs = "https://esgf-node.llnl.gov/solr/datasets/select?q=*:*&rows=0&wt=json&facet=true" \
#             "&fq=type:Dataset&fq=replica:false&fq=project:input4MIPs&facet.field=source_id"
#js = requests.get(input4MIPs)
#js_MIPs = json.loads(js.text)
#print(input4MIPs)

#input4mips = "https://esgf-node.llnl.gov/solr/datasets/select?q=*:*&rows=0&wt=json&facet=true" \
#             "&fq=type:Dataset&fq=replica:false&fq=project:input4mips&facet.field=source_id"
#js = requests.get(input4mips)
#js_mips = json.loads(js.text)
#print(input4mips)

inputs = "https://esgf-node.llnl.gov/solr/datasets/select?q=*:*&rows=0&wt=json&facet=true&" \
         "fq=type:Dataset&fq=replica:false&fq=project:(input4mips%20OR%20input4MIPs)&facet.field=source_id"
js = requests.get(inputs)
js_mips = json.loads(js.text)
print(inputs)

https://esgf-node.llnl.gov/solr/datasets/select?q=*:*&rows=0&wt=json&facet=true&fq=type:Dataset&fq=replica:false&fq=project:(input4mips%20OR%20input4MIPs)&facet.field=source_id


## extract source_id entries and composite

In [55]:
#print(js_MIPs)
#print(js_MIPs.keys())
#print(js_mips["facet_counts"]["facet_fields"]["source_id"])
srcIds = js_mips["facet_counts"]["facet_fields"]["source_id"]
srcIdLen = len(srcIds)
print("srcIdLen:", srcIdLen)
els = np.arange(0, srcIdLen, 2)
#print(els.tolist())
srcIdDict = {}
counts = 0
for cnt, srcId in enumerate(els.tolist()):
    #print(cnt, srcId)
    srcIdDict[srcIds[srcId]] = srcIds[srcId+1]
    counts += int(srcIds[srcId+1])

print("len(srcIdDict.keys()):", len(srcIdDict.keys()))
print("dataset counts:", counts)
print(srcIdDict)

srcIdLen: 368
len(srcIdDict.keys()): 184
dataset counts: 5876
{'ACCESS1-3-rcp85-1-0': 4, 'CCMI-hist-nat-1-0': 1, 'CCMI-hist-nat-1-1': 1, 'CCMI-hist-sol-1-0': 1, 'CCMI-hist-sol-1-1': 1, 'CCMI-hist-stratO3-1-0': 1, 'CCMI-hist-volc-1-0': 1, 'CCMI-hist-volc-1-1': 1, 'CCMI-ssp245-nat-1-0': 1, 'CCMI-ssp245-sol-1-0': 1, 'CCMI-ssp245-stratO3-1-0': 1, 'CCMI-ssp245-volc-1-0': 1, 'CCSM4-rcp26-1-0': 5, 'CCSM4-rcp85-1-0': 8, 'CEDS-2016-06-18': 7, 'CEDS-2016-06-18-sectorDimV2': 37, 'CEDS-2016-06-18-supplemental-data': 30, 'CEDS-2016-07-26': 7, 'CEDS-2016-07-26-sectorDim': 7, 'CEDS-2016-07-26-sectorDim-supplemental-data': 30, 'CEDS-2017-05-18': 18, 'CEDS-2017-05-18-supplemental-data': 33, 'CEDS-2017-08-30': 9, 'CEDS-2017-08-30-supplemental-data': 1, 'CEDS-2017-10-05': 1, 'CESM2-ssp585-1-0': 12, 'CNRM-CM6-1-ssp126-1-0': 12, 'CNRM-CM6-1-ssp585-1-0': 12, 'CNRM-ESM2-1-ssp585-1-0': 12, 'CSIRO-MK3-6-0-rcp85-1-0': 11, 'DCPP-C-amv-1-1': 4, 'DCPP-C-ipv-1-1': 3, 'HadGEM2-ES-rcp85-1-0': 12, 'IACETH-SAGE3lambda-

## example source_id queries

In [ ]:
https://esgf-node.llnl.gov/esg-search/search/?limit=0&format=application%2Fsolr%2Bjson&facets=variable_id&project=input4mips&project=input4MIPs&mip_era=CMIP6&distrib=false